# Import libraries and functions

In [1]:
import math
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

## Initialize Orekit and import Orekit libraries

In [3]:
import orekit
vm = orekit.initVM()
print ('Java version:',vm.java_version)
print ('Orekit version:', orekit.VERSION)

Java version: 1.8.0_382
Orekit version: 12.0.1


In [4]:
from orekit.pyhelpers import setup_orekit_curdir, download_orekit_data_curdir
setup_orekit_curdir('../../orekit-data.zip')

In [5]:
from java.util import Arrays
from orekit import JArray_double

In [6]:
from org.orekit.propagation.analytical.tle import TLE, TLEPropagator
from org.orekit.frames import FramesFactory, LOFType, LocalOrbitalFrame

# Load Space-Track dataset

In [55]:
reduced, frac = True, 0.25
reduced_sample_alt_e, min_alt, max_alt, e_thres = False, 500, 600, 0.2
if reduced:
    filepath = f"../datasets/space-track-dataset-reduced-{int(frac*100)}.csv"
elif reduced_sample_alt_e:
    filepath = f"../datasets/space-track-dataset-reduced-{int(frac*100)}-h-{min_alt}-{max_alt}-e-{int(e_thres*100)}.csv"
else:
     filepath = '../datasets/space-track-dataset.csv'

df = pd.read_csv(filepath, nrows=101 ,memory_map=True)
df.head()

,NORAD_CAT_ID,OBJECT_NAME,OBJECT_ID,DECAY_DATE,EPOCH_DATE,EPOCH_TIME,CREATION_DATE,CENTER_NAME,REF_FRAME,TIME_SYSTEM,...,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS,OBJECT_TYPE,RCS_SIZE,TLE_LINE1,TLE_LINE2
0,58063,STARLINK-30585,2023-158N,NaN,2023-12-28,00:00:00.999648,2023-12-28T06:16:17,EARTH,TEME,UTC,...,-1.212631e-02,0.0,6934.593,95.784,557.373,555.544,PAYLOAD,LARGE,1 58063U 23158N 23362.00001157 -.01212631 0...,2 58063 43.0014 71.6139 0001319 262.3570 310...
1,45739,STARLINK-1475,2020-038K,NaN,2023-12-28,00:01:55.626528,2023-12-28T18:10:27,EARTH,TEME,UTC,...,8.730000e-06,0.0,6925.347,95.592,548.089,546.334,PAYLOAD,LARGE,1 45739U 20038K 23362.00133827 .00000873 0...,2 45739 53.0535 200.0197 0001267 97.3550 262...
2,45227,STARLINK-1221,2020-012BB,NaN,2023-12-28,00:01:58.804320,2023-12-28T06:26:17,EARTH,TEME,UTC,...,1.595000e-05,0.0,6925.400,95.593,548.114,546.416,PAYLOAD,LARGE,1 45227U 20012BB 23362.00137505 .00001595 0...,2 45227 53.0556 10.0261 0001226 101.0635 259...
3,27609,TRAILBLAZER 2,2002-058E,NaN,2023-12-28,00:02:24.040032,2023-12-28T06:16:17,EARTH,TEME,UTC,...,6.300000e-06,0.0,7009.008,97.330,638.150,623.597,PAYLOAD,LARGE,1 27609U 02058E 23362.00166713 .00000630 0...,2 27609 64.5552 236.5835 0010382 181.6066 178...
4,14879,THORAD DELTA 1 DEB,1974-089ES,NaN,2023-12-28,00:02:43.493856,2023-12-28T18:10:27,EARTH,TEME,UTC,...,5.300000e-07,0.0,7990.496,118.473,1783.498,1441.225,DEBRIS,SMALL,1 14879U 74089ES 23362.00189229 .00000053 0...,2 14879 101.2066 12.9018 0214175 298.1690 85...


## Remove fractional part of seconds in EPOCH_TIME

In [56]:
df['EPOCH_TIME'] = df['EPOCH_TIME'].apply(lambda t: t.split('.')[0])
df['EPOCH_TIME'].head()

0    00:00:00
1    00:01:55
2    00:01:58
3    00:02:24
4    00:02:43
Name: EPOCH_TIME, dtype: object

# Build Space-Track dynamic satellite graph from 2023-12-28 to 2024-01-28

In [115]:
def tle_to_pv(tle_line1, tle_line2, date=None, transformer=None):
    tle = TLE(tle_line1, tle_line2)
    propagator = TLEPropagator.selectExtrapolator(tle)
    
    if date is None:
        state = propagator.getInitialState()
    else:
        state = propagator.propagate(date)
        
    if transformer is None:
        initial_frame = state.getFrame()
        lof = LocalOrbitalFrame(initial_frame, LOFType.QSW, propagator,str(t.getSatelliteNumber())+"_lof")
        transformer = initial_frame.getTransformTo(lof, tle.getDate())
        
    pv = state.getPVCoordinates()
    return transformer.transformPVCoordinates(pv), transformer

In [116]:
def conjunction(pv1, pv2, limits):
    distance = -1
    (r_lim, r_weight), (it_lim, it_weight), (ct_lim, ct_weight) = limits
    pos1 = pv1.getPosition()
    pos2 = pv2.getPosition()
    radial1, in_track1, cross_track1 = pos1.x, pos1.y, pos1.z 
    radial2, in_track2, cross_track2 = pos2.x, pos2.y, pos2.z
    r_dist = math.fabs(radial1-radial2)
    it_dist = math.fabs(in_track1-in_track2)
    ct_dist = math.fabs(cross_track1-cross_track2)
    
    if r_weight*r_dist <= r_weight*r_lim or it_weight*it_dist <= it_weight*it_lim or ct_weight*ct_dist <= ct_weight*ct_lim:
        distance = r_weight*(r_dist**2) + it_weight*(it_dist**2) + ct_weight*(ct_dist**2)
    
    return r_dist, it_dist, ct_dist, math.sqrt(distance) if distance >= 0 else distance

In [117]:
def edge_exists(id1, id2, edges):
    src_tgt_pairs = list(zip(edges['source'], edges['target']))
    
    exists_src_tgt = (id1, id2) in src_tgt_pairs
    exists_tgt_src = (id2, id1) in src_tgt_pairs
    index = -1
    # try except with src_tgt_pairs.index((id1,id2))
    if exists_src_tgt or exists_tgt_src:
        index = next((i for i, src_tgt in enumerate(src_tgt_pairs) if ((id1, id2) == src_tgt) or ((id2, id1) == src_tgt)), -1)
    
    return index

In [118]:
def total_seconds(str_time):
    t = str_time.split(':')
    h, m, s = int(t[0]), int(t[1]), int(t[2])
    return (h * 3600) + (m * 60) + s

In [119]:
def update_edges_data(edges_data, data, idx=None):
    if idx is None:
        edges_data['source'].append(data['source'])
        edges_data['target'].append(data['target'])
        edges_data['weight'].append(data['weight'])
        edges_data['r_dist'].append(data['r_dist'])
        edges_data['it_dist'].append(data['it_dist'])
        edges_data['ct_dist'].append(data['ct_dist'])
        edges_data['dist'].append(data['dist'])
        edges_data['time'].append(data['time'])
        edges_data['prop'].append(data['prop'])
    else:
        edges_data['weight'][idx]+=1
        edges_data['r_dist'][idx].append(data['r_dist'])
        edges_data['it_dist'][idx].append(data['it_dist'])
        edges_data['ct_dist'][idx].append(data['ct_dist'])
        edges_data['dist'][idx].append(data['dist'])
        edges_data['time'][idx].append(data['time'])
        edges_data['prop'][idx].append(data['prop'])

In [209]:
def check_conjunction(edges_data, time, prop, sat1_id, pv1_lof1, sat2_id, pv2_lof1, limits):
    r_dist, it_dist, ct_dist, distance = conjunction(pv1_lof1, pv2_lof1, limits)
    if distance >= 0:
        idx = edge_exists(sat1_id, sat2_id, edges_data)
        if idx >= 0:
            data_update = {'r_dist':r_dist, 'it_dist':it_dist, 'ct_dist':ct_dist, 'dist':distance, 'time':time, 'prop':prop}
            update_edges_data(edges_data, data_update, idx=idx)
        else:
            data_update = {'source':sat1_id, 'target': sat2_id, 'weight':1,'r_dist':[r_dist], 'it_dist':[it_dist], 'ct_dist':[ct_dist], 'dist':[distance], 'time':[time], 'prop':[prop]}
            update_edges_data(edges_data, data_update)

In [349]:
def sat_data(data, sat_id, time=None):
    satellite_data = data[data[:, 0] == sat_id]
    idx = 0
    if time is not None:
        # closest timestamp to time idx
        idx = np.argmin(np.abs(np.vectorize(total_seconds)(satellite_data[:, 3]) - total_seconds(time)))
    return sat_id, satellite_data[idx, 1], satellite_data[idx, 2], satellite_data[idx, 3]

In [350]:
def sat2_conjunctions(edges_data, time, data, sat1_id, pv1_lof1, transform1, sat2_ids, limits, prop):
    for sat2_id in sat2_ids: 
        sat2_id, sat2_tle_line1, sat2_tle_line2, sat2_time = sat_data(data, sat2_id, time if prop else None)
    
        pv2_lof1, _ = tle_to_pv(sat2_tle_line1, sat2_tle_line2, date=pv1_lof1.getDate() if prop else None, transformer=transform1)
        check_conjunction(edges_data, time, prop, sat1_id, pv1_lof1, sat2_id, pv2_lof1, limits)

In [360]:
# Check for conjunctions between satellites observed at time
def conjunctions_at_timestamp(edges_data, sat1_data_idx, time, data, limits):
    for idx1 in sat1_data_idx:
        sat1_data = data[idx1]
        sat1_id, sat1_tle_line1, sat1_tle_line2, sat1_time = sat1_data[0], sat1_data[1], sat1_data[2], sat1_data[3]
        
        pv1_lof1, transform1 = tle_to_pv(sat1_tle_line1, sat1_tle_line2)
        
        idx_idx1 = np.where(sat1_data_idx == idx1)[0][0]
        sat2_idx = sat1_data_idx[idx_idx1+1:] if idx_idx1 < sat1_data_idx.shape[0]-1 else []
        sat2_ids = data[sat2_idx, 0]

        sat2_conjunctions(edges_data, time, data, sat1_id, pv1_lof1, transform1, sat2_ids, limits, False)
    return edges_data

In [361]:
# Check for conjunctions between satellites observed at time and satellites from other timestamps of date
def conjunctions_after_propagation(edges_data, sat1_data_idx, time, data, ids, sat1_ids, limits):
    for idx1 in sat1_data_idx:
        sat1_data = data[idx1]
        sat1_id, sat1_tle_line1, sat1_tle_line2, sat1_time = sat1_data[0], sat1_data[1], sat1_data[2], sat1_data[3]

        pv1_lof1, transform1 = tle_to_pv(sat1_tle_line1, sat1_tle_line2)
        
        sat2_ids = ids[np.isin(ids, sat1_ids, invert=True)]
        sat2_conjunctions(edges_data, time, data, sat1_id, pv1_lof1, transform1, sat2_ids, limits, True)
    return edges_data

In [362]:
def satellite_graph_edges(edges_data, ids, date, time, data, limits):
    timestamp_data = data[:, 3] == time
    sat1_data_idx = np.where(timestamp_data)[0]
    
    edges_data = conjunctions_at_timestamp(edges_data, sat1_data_idx, time, data, limits)
    
    sat1_ids = data[sat1_data_idx][:, 0]
    
    edges_data = conjunctions_after_propagation(edges_data, sat1_data_idx, time, data, ids, sat1_ids, limits)
    
    edges_data['date'] = [date] * len(edges_data['source'])
    return edges_data

In [363]:
def tle_to_edges(tle_df, dates, limits):
    edges = {'source':[], 'target':[], 'weight':[], 'r_dist':[], 'it_dist':[], 'ct_dist':[], 'dist':[], 'date':[], 'time':[], 'prop':[]}
    for date in tqdm(dates):
        tle_df_date = tle_df[tle_df['EPOCH_DATE'] == date]
        ids = tle_df_date['NORAD_CAT_ID'].unique()
        data = tle_df_date[['NORAD_CAT_ID', 'TLE_LINE1', 'TLE_LINE2', 'EPOCH_TIME']].to_numpy()
        timestamps = tle_df_date['EPOCH_TIME'].unique()
        edges_data = {'source':[], 'target':[], 'weight':[], 'r_dist':[], 'it_dist':[], 'ct_dist':[], 'dist':[], 'date':[], 'time':[], 'prop':[]}
        for time in tqdm(timestamps):
            edges_data = satellite_graph_edges(edges_data, ids, date, time, data, limits)
            
        edges['source'] = edges['source'] + edges_data['source']
        edges['target'] = edges['target'] + edges_data['target']
        edges['weight'] = edges['weight'] + edges_data['weight']
        edges['r_dist'] = edges['r_dist'] + edges_data['r_dist']
        edges['it_dist'] = edges['it_dist'] + edges_data['it_dist']
        edges['ct_dist'] = edges['ct_dist'] + edges_data['ct_dist']
        edges['dist'] = edges['dist'] + edges_data['dist']
        edges['date'] = edges['date'] + edges_data['date']
        edges['time'] = edges['time'] + edges_data['time']
        edges['prop'] = edges['prop'] + edges_data['prop']
    return pd.DataFrame(edges)

In [364]:
dates = df['EPOCH_DATE'].unique()
dates

array(['2023-12-28'], dtype=object)

In [365]:
def to_m(km):
    return km * 1000

In [366]:
leo1_limits = (0.4, 44, 51) 
leo2_limits = (0.4, 25, 25)
leo3_limits = (0.4, 12, 12)
leo4_limits = (0.4, 2, 2)

In [367]:
limit_weights = (1.0, 1.0, 1.0)
limits = tuple(map(to_m, leo1_limits))
edges_df = tle_to_edges(df, dates, tuple(zip(limits, limit_weights)))
edges_df.head()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

,source,target,weight,r_dist,it_dist,ct_dist,dist,date,time,prop
0,58063,25403,1,[8272396.474554408],[50.742606117390096],[7324950.436896488],[11049318.636811279],2023-12-28,[00:00:00],[True]
1,58063,51881,1,[1812052.1585625857],[4655556.537659448],[35257.95526932902],[4995896.598620185],2023-12-28,[00:00:00],[True]
2,58063,52603,1,[5669566.660333483],[16240.69561928697],[6807025.431088046],[8858885.093263065],2023-12-28,[00:00:00],[True]
3,45739,53982,2,"[210945.99666316714, 197783.10838391283]","[8040.341389375739, 4993.024602268124]","[1670487.782854712, 1668337.2989635281]","[1683773.2309502745, 1680027.5091109676]",2023-12-28,"[00:01:55, 00:04:40]","[True, True]"
4,45739,43073,1,[11628060.816076959],[6380.830997496843],[5402891.381673055],[12821976.225978019],2023-12-28,[00:01:55],[True]


In [368]:
edges_df['weight'].describe()

count    188.000000
mean       1.287234
std        0.453680
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max        2.000000
Name: weight, dtype: float64

In [369]:
src_nodes = edges_df['source'].unique()
src_nodes.sort()

tgt_nodes = edges_df['target'].unique()
tgt_nodes.sort()

num_common_nodes = np.intersect1d(src_nodes, tgt_nodes).shape[0]
num_total_nodes = num_common_nodes + (src_nodes.shape[0] - num_common_nodes) + (tgt_nodes.shape[0] - num_common_nodes)

print(f'Total number of unique nodes: {df["NORAD_CAT_ID"].unique().shape[0]}')
print(f'Number of unique source nodes: {src_nodes.shape[0]}')
print(f'Number of unique target nodes: {tgt_nodes.shape[0]}')
print(f'Number of unique source and target nodes: {num_common_nodes}')
print(f'Number of edges: {edges_df.shape[0]}')
print(f'Graph density: {edges_df.shape[0] / (num_total_nodes*(num_total_nodes-1)):.2f}')

Total number of unique nodes: 101
Number of unique source nodes: 87
Number of unique target nodes: 80
Number of unique source and target nodes: 68
Number of edges: 188
Graph density: 0.02


In [ ]:
if reduced:
    savepath = f"../datasets/space-track-ap1-graph-edges-reduced-{int(frac*100)}.csv"
elif reduced_sample_alt_e:
    savepath = f"../datasets/space-track-ap1-graph-edges-reduced-{int(frac*100)}-h-{min_alt}-{max_alt}-e-{int(e_thres*100)}.csv"
else:
    savepath = '../datasets/space-track-ap1-graph-edges.csv'
edges_df.to_csv(savepath, index=False)